<a href="https://colab.research.google.com/github/toby-brett/Stanford-CS229-projects/blob/main/GLM1_Binary_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [197]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

data = pd.read_csv('/content/drive/My Drive/mushroom/agaricus-lepiota.data').head(8123)
test = pd.read_csv('/content/drive/My Drive/mushroom/agaricus-lepiota.data').head(8123)
print(data.head())

# poisonous	Target				      poisionus=p, not=e

# cap-shape			                bell=b,conical=c,convex=x,flat=f, knobbed=k,sunken=s
# cap-surface			              fibrous=f,grooves=g,scaly=y,smooth=s
# cap-color			                brown=n,buff=b,cinnamon=c,gray=g,green=r, pink=p,purple=u,red=e,white=w,yellow=y
# bruises			                  bruises=t,no=f
# odor			                    almond=a,anise=l,creosote=c,fishy=y,foul=f, musty=m,none=n,pungent=p,spicy=s
# gill-attachment			          attached=a,descending=d,free=f,notched=n
# gill-spacing			            close=c,crowded=w,distant=d
# gill-size			                broad=b,narrow=n
# gill-color			              black=k,brown=n,buff=b,chocolate=h,gray=g, green=r,orange=o,pink=p,purple=u,red=e, white=w,yellow=y		no
# stalk-shape			              enlarging=e,tapering=t
# stalk-root			              bulbous=b,club=c,cup=u,equal=e, rhizomorphs=z,rooted=r,missing=?
# stalk-surface-above-ring			fibrous=f,scaly=y,silky=k,smooth=s
# stalk-surface-below-ring			fibrous=f,scaly=y,silky=k,smooth=s
# stalk-color-above-ring			  brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
# stalk-color-below-ring			  brown=n,buff=b,cinnamon=c,gray=g,orange=o, pink=p,red=e,white=w,yellow=y
# veil-type			                partial=p,universal=u
# veil-color			              brown=n,orange=o,white=w,yellow=y
# ring-number			              none=n,one=o,two=t
# ring-type			                cobwebby=c,evanescent=e,flaring=f,large=l, none=n,pendant=p,sheathing=s,zone=z
# spore-print-color			        black=k,brown=n,buff=b,chocolate=h,green=r, orange=o,purple=u,white=w,yellow=y
# population			              abundant=a,clustered=c,numerous=n, scattered=s,several=v,solitary=y
# habitat			                  grasses=g,leaves=l,meadows=m,paths=p,urban=u,waste=w,woods=d¢

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
   p  x  s  n  t p.1  f  c n.1  k  ... s.2  w w.1 p.2 w.2  o p.3 k.1 s.3  u
0  e  x  s  y  t   a  f  c   b  k  ...   s  w   w   p   w  o   p   n   n  g
1  e  b  s  w  t   l  f  c   b  n  ...   s  w   w   p   w  o   p   n   n  m
2  p  x  y  w  t   p  f  c   n  n  ...   s  w   w   p   w  o   p   k   s  u
3  e  x  s  g  f   n  f  w   b  k  ...   s  w   w   p   w  o   e   n   a  g
4  e  x  y  y  t   a  f  c   b  n  ...   s  w   w   p   w  o   p   k   n  g

[5 rows x 23 columns]


In [199]:
data_encoded = pd.get_dummies(data)
test_data_encoded = pd.get_dummies(test)

data_encoded = data_encoded.astype(int)
test_data_encoded = test_data_encoded.astype(int)

y_train_df = data_encoded.iloc[:, 2] # uses p_p as target, 1=poison, 0=benign
X_train_df = data_encoded.iloc[:, 2:]

y_test_df = test_data_encoded.iloc[:, 2]
X_test_df = test_data_encoded.iloc[:, 2:]

print(y_train_df.head()) # gets the p_e = 0 if benign and p_p = 1 if poision. For all datatypes

0    0
1    1
2    0
3    0
4    0
Name: x_b, dtype: int64


In [200]:
import numpy as np

X_train = np.array(X_train_df)[0:8000, :]
y_train = np.array(y_train_df)[0:8000]

X_test = np.array(X_test_df)[8000:8123, :]
y_test = np.array(y_test_df)[8000:8123]

print(X_train.shape)
print(X_test.shape)

(8000, 117)
(123, 117)


In [201]:
import math

# create exp fam class

class BernouliExpFam:
  def __init__(self):
    pass
  def b(self, y): # base measure
    return 1
  def a(self, eta): # log partition function
    return -math.log(1 + math.exp(eta))
  def T(self, y): # sufficient statistic
    return
  def phi(self, eta):
    return 1 / (1 + math.exp(-eta))

In [202]:
# create the GLM

exp_family = BernouliExpFam()

class GLM:
  def __init__(self):
    self.alpha = 0.01
    self.theta = np.zeros((1, 117)) # already transposed

  def forward(self, x):
    self.eta = self.theta @ x
    self.pred = exp_family.phi(self.eta) # takes mean (which is phi and also logistic function)
    return self.pred

  def update(self, x, y):
    self.theta = self.theta + self.alpha * (y - self.forward(x)) * x # update rule (check notes)


In [203]:
model = GLM()

for example, label in zip(X_train, y_train):
  model.update(example, label)


<ipython-input-201-72932add8d96>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-eta))


In [204]:
correct = 0

for example, label in zip(X_test, y_test):
  if round(model.forward(example)) == label:
    correct +=1

print(correct/123) # every test example correct holy moly

1.0


<ipython-input-201-72932add8d96>:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-eta))
